<a href="https://colab.research.google.com/github/WoradeeKongthong/ComputerVision/blob/master/CV_Model_Architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG Blocks

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
from tensorflow.keras.utils import plot_model
from IPython.display import Image 
import matplotlib.pyplot as plt

In [3]:
# function for creating a vgg block
def vgg_block(layer_in, n_filters, n_conv):
	# add convolutional layers
	for _ in range(n_conv):
		layer_in = Conv2D(n_filters, (3,3), padding='same', activation='relu')(layer_in)
	# add max pooling layer
	layer_in = MaxPooling2D((2,2), strides=(2,2))(layer_in)
	return layer_in

In [4]:
# create model with Functional API

# define model input
visible = Input(shape=(256, 256, 3))

# add 1 vgg module
layer = vgg_block(visible, 64, 2)

# create model
model = Model(inputs=visible, outputs=layer)

# summarize model
model.summary()

# plot model architecture
plot_model(model, show_shapes=True)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
Total params: 38,720
Trainable params: 38,720
Non-trainable params: 0
_________________________________________________________________
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [5]:
# create model (3 VGG blocks)
# define model input
visible = Input(shape=(256, 256, 3))

# add 1 vgg module
layer = vgg_block(visible, 64, 2)
layer = vgg_block(layer, 128, 2)
layer = vgg_block(layer, 256, 4)

# create model
model = Model(inputs=visible, outputs=layer)

# summarize model
model.summary()

# plot model architecture
plot_model(model, show_shapes=True)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 128)       0   

# Naive Inception Module
A block of parallel convolutional layers with 'different' sized filters (e.g. 1x1, 3x3, 5x5)  
and a 3x3 max pooling layer,  
the results of which are then concatenated.


In [2]:
from tensorflow.keras.layers import concatenate

In [3]:
# function for creating a naive inception block
def naive_inception_module(layer_in, f1, f2, f3):
	# 1x1 conv
	conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
	# 3x3 conv
	conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(layer_in)
	# 5x5 conv
	conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(layer_in)
	# 3x3 max pooling
	pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
	# concatenate filters, assumes filters/channels last
	layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
	return layer_out

In [4]:
# define model input
visible = Input(shape=(256, 256, 3))

# add inception module
layer = naive_inception_module(visible, 64, 128, 32)

# create model
model = Model(inputs=visible, outputs=layer)

# summarize model
model.summary()

# plot model architecture
plot_model(model, show_shapes=True, to_file='naive_inception_module.png')

NameError: name 'Input' is not defined

# Optimized Inception Module
To use many inception modules in your model, you need to **control number of filters**
*   prior to the 3x3 conv
*   prior to the 5x5 conv
*   after max pooling

In [9]:
# function for creating a projected inception module
def inception_module(layer_in, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
	# 1x1 conv
	conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
	# 3x3 conv
	conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu')(layer_in)
	conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu')(conv3)
	# 5x5 conv
	conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu')(layer_in)
	conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu')(conv5)
	# 3x3 max pooling
	pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
	pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
	# concatenate filters, assumes filters/channels last
	layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
	return layer_out

In [10]:
# define model input
visible = Input(shape=(256, 256, 3))

# add inception block 1
layer = inception_module(visible, 64, 96, 128, 16, 32, 32)

# add inception block 2
layer = inception_module(layer, 128, 128, 192, 32, 96, 64)

# create model
model = Model(inputs=visible, outputs=layer)

# summarize model
model.summary()

# plot model architecture
plot_model(model, show_shapes=True, to_file='inception_module.png')

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 256, 256, 96) 384         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 256, 256, 16) 64          input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 256, 256, 3)  0           input_4[0][0]                    
____________________________________________________________________________________________

# Residual Module
The output of the second layer is added with the input to the first conv layer.  
Called a shortcut connection.

In [5]:
from tensorflow.keras.layers import Activation, add

In [12]:
# function for creating a residual module
def residual_module(layer_in, n_filters):
	merge_input = layer_in
	# check if the number of filters needs to be increase
	if layer_in.shape[-1] != n_filters:
		merge_input = Conv2D(n_filters, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	
  # conv1
	conv1 = Conv2D(n_filters, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# conv2
	conv2 = Conv2D(n_filters, (3,3), padding='same', activation='linear', kernel_initializer='he_normal')(conv1)
	
  # add first layer input to second layer output
	layer_out = add([conv2, merge_input])
 
	# activation function
	layer_out = Activation('relu')(layer_out)
 
	return layer_out

In [13]:
# define model input
visible = Input(shape=(256, 256, 3))

# add vgg module
layer = residual_module(visible, 64)

# create model
model = Model(inputs=visible, outputs=layer)

# summarize model
model.summary()

# plot model architecture
plot_model(model, show_shapes=True, to_file='residual_module.png')

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 256, 256, 64) 1792        input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_26[0][0]                  
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 256, 256, 64) 256         input_5[0][0]                    
____________________________________________________________________________________________